## separate label and images after labelimg tool 

In [1]:
import os
import shutil

In [2]:
xml_list = [file for file in os.listdir('test/') if file.endswith('.xml')]
print(len(xml_list))
label_dir ='label/'
if not os.path.exists(label_dir):
        os.mkdir(label_dir)

28


In [3]:
for i in xml_list:
    shutil.move('test/'+i, 'label/'+i)

#### convert  ICDAR format from VOC format

In [4]:
from pathlib import Path
import xml.etree.ElementTree as ET
from typing import Dict
import os
import shutil

class XMLHandler:
    def __init__(self, xml_path: str or Path):
        self.xml_path = Path(xml_path)
        self.root = self.__open()

    def __open(self):
        with self.xml_path.open() as opened_xml_file:
            self.tree = ET.parse(opened_xml_file)
            return self.tree.getroot()

    def return_boxes_class_as_dict(self) -> Dict[int, Dict]:
        boxes_dict = {}
        for index, sg_box in enumerate(self.root.iter('object')):
            boxes_dict[index] = {"name": sg_box.find("name").text,
                                 "xmin": int(sg_box.find("bndbox").find("xmin").text),
                                 "ymin": int(sg_box.find("bndbox").find("ymin").text),
                                 "xmax": int(sg_box.find("bndbox").find("xmax").text),
                                 "ymax": int(sg_box.find("bndbox").find("ymax").text)}

        return boxes_dict


def converter(xml_files: str, output_folder: str) -> None:
    xml_files = [file for file in os.listdir('label') if file.endswith('.xml')]

    for xml_index, xml in enumerate(xml_files):
        print(xml)
        file_name = xml[:-4]
        filename = "{}.txt".format(file_name)
        
        filename_path = Path(output_folder) / filename
        xml_content = XMLHandler('label/'+xml)
        boxes = xml_content.return_boxes_class_as_dict()

        with open(filename_path, "a") as file:
            for box_index in boxes:
                box = boxes[box_index]
                if box['name']=='-':
                    continue
                print(box['name'])
                box_content = f"{box['name']} {box['xmin']} {box['ymin']} {box['xmax']} {box['ymax']}\n"
                file.write(box_content)

    print(f"Converted {len(xml_files)} files!")

    
if __name__ == '__main__':
    XML_FOLDER = "label"
    OUTPUT_FOLDER =  "icdar"
    if not os.path.exists('icdar/'):
        os.mkdir('icdar/')
    converter(XML_FOLDER, OUTPUT_FOLDER)

PB150007.xml
VU936E
P9170033.xml
CK425AB
PB150009.xml
PU826DL
P9170011.xml
ZG8524AF
P9170042.xml
ZG2751B
P9170013.xml
ST377IE
PB150005.xml
VU279AE
P9170019.xml
HAD677
P9170051.xml
G22SKD
PB150001.xml
SB797AM
P9170054.xml
W78512C
P9170053.xml
SPG065
PB150008.xml
ZG7341H
P9170012.xml
ZG593ZH
PB150003.xml
A984KR
PB150004.xml
KEHH241
P9170017.xml
DU986BM
PB150011.xml
VU727AK
P9170024.xml
KA570CD
P9170052.xml
MMM7780
P9170008.xml
ZG3254D
P9170014.xml
SK314CK
PB150012.xml
ZG2563AC
P9170023.xml
SK553AL
PB150002.xml
ZG794VM
PB150010.xml
VU727AK
P9170018.xml
RI238JL
PB150006.xml
VU936E
Converted 28 files!


#### CRNN format changing

In [2]:
file_list = [file for file in os.listdir('icdar') if file.endswith('.txt')]
print(len(file_list))
if not os.path.exists('images/'):
        os.mkdir('images/')

450


In [3]:
n = 1
from PIL import Image
with open('train.txt','w') as f:
    for i in file_list:
        img_name = i[:-4]
        name = img_name+'.jpg'
        with open('icdar/'+i,'r') as a:
            lines = a.read().splitlines()
            for j in lines:
                line = j.split(' ')
                im = Image.open('JPEGImages/'+name)
                area = (int(line[1]), int(line[2]), int(line[3]), int(line[4]))
                cropped_im = im.crop(area)
                final_name = 'images/{}_{}.jpg'.format(line[0],str(n))
                cropped_im.save(final_name, quality=100, optimize=True)
                f.write('{}{}{}'.format(final_name,' ',line[0]))
                f.write('\n')
                n=n+1

### data augmentation for bus number detection

In [1]:
import numpy as np
import os
import cv2

In [2]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [3]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [4]:
def blur(src):
    dst = cv2.GaussianBlur(src,(5,5),cv2.BORDER_DEFAULT)
    return dst

In [5]:
def contrast(img):
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)

    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

In [6]:
def motion_horizontal_blur(img):
    size = randrange(1, 5)
    # generating the horizontal kernel
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size

    # applying the kernel to the input image
    output = cv2.filter2D(img, -1, kernel_motion_blur)
    return output

In [8]:
# def motion_vertical_blur(img):
#     size = randrange(1, 5)
#     kernel_motion_blur = np.zeros((size, size))
#     kernel_motion_blur[int(:, (size-1)/2)] = np.ones(size)
#     kernel_motion_blur = kernel_motion_blur / size
#     # applying the kernel to the input image
#     output = cv2.filter2D(img, -1, kernel_motion_blur)
#     return output

In [9]:
if not os.path.exists('daimg/'):
    os.mkdir('daimg/')
if not os.path.exists('dalabel/'):
    os.mkdir('dalabel/')

In [10]:
imagelist = [file for file in os.listdir('JPEGImages/') if file.endswith('.jpg')]
print(len(imagelist))

450


In [12]:
import numpy as np
from random import randrange, uniform
num = 1
for img in imagelist:
    img_name = img[:-4]
    img = cv2.imread("JPEGImages/{}".format(img))
    txt_name = img_name+'.txt'
    # adding noise
    blur_img=True
    brightness =True
    noise = True
    contrast_ratio =True
    motion_blur_hori=True
    if motion_blur_hori:
        motion_img = motion_horizontal_blur(img)
        with open('dalabel/'+str(num)+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}{}".format(num,'.jpg'),motion_img)
                num=num+1
    if contrast_ratio:
        cont_img =contrast(img)
        with open('dalabel/'+str(num)+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}{}".format(num,'.jpg'),cont_img)
                num=num+1
    if brightness:  
        bri = increase_brightness(img)     
        with open('dalabel/'+str(num)+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}{}".format(num,'.jpg'),bri)
                num=num+1
    if blur_img:  
        b_img = blur(img)     
        with open('dalabel/'+str(num)+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}{}".format(num,'.jpg'),b_img)
                num=num+1
            
    if noise:
        noi = ['s&p','gauss','s&p']
        n = np.random.choice(noi,1,replace = False)
        noise_sp = noisy(n,img)
        with open('dalabel/'+str(num)+'.txt','w') as df:
            with open('icdar/'+txt_name,'r') as nf:
                line = nf.read()
                df.write(line)
                cv2.imwrite("daimg/{}{}".format(num,'.jpg'),noise_sp)
                num=num+1   

<ipython-input-3-579615bd4155>:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 1
<ipython-input-3-579615bd4155>:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 0


#### data augmentation for bus number recognition

In [13]:
import numpy as np
import os
import cv2
import numpy as np
from random import randrange, uniform

In [5]:
crop_images = 'images/'
imagelist = [file for file in os.listdir(crop_images) if file.endswith('.jpg')]
print(len(imagelist))

450


In [6]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [7]:
def motion_horizontal_blur(img):
    size = randrange(1, 5)
    # generating the horizontal kernel
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size

    # applying the kernel to the input image
    output = cv2.filter2D(img, -1, kernel_motion_blur)
    return output

In [8]:
def contrast(img):
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 0 # Brightness control (0-100)

    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

In [9]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [10]:
def blur(src):
    dst = cv2.GaussianBlur(src,(5,5),cv2.BORDER_DEFAULT)
    return dst

In [11]:
if not os.path.exists('R_daimg/'):
    os.mkdir('R_daimg/')

In [14]:
num = 1
with open('train_da.txt','w') as df:
    with open('train.txt','r') as rd:
        abc = rd.read().splitlines()       
        for img1 in abc:
            img = img1.split(' ')
            print(img[0])
            ab =img[1]
            img = cv2.imread("{}".format(img[0]))
            blur_img=True
            brightness =True
            noise = True
            contrast_ratio =True
            motion_blur_hori=True
            if motion_blur_hori:
                motion_img = motion_horizontal_blur(img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),motion_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if contrast_ratio:
                cont_img =contrast(img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),cont_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if brightness:  
                bri = increase_brightness(img)     
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),bri)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1
            if blur_img:  
                b_img = blur(img)     
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),b_img)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1

            if noise:
                noi = ['s&p','gauss','s&p']
                n = np.random.choice(noi,1,replace = False)
                noise_sp = noisy(n,img)
                cv2.imwrite("R_daimg/{}{}".format(num,'.jpg'),noise_sp)
                df.write("images/{}{}{}{}".format(num,'.jpg',' ',ab))
                df.write('\n')
                num=num+1   

images/ZG4940L_1.jpg
images/BJ286BM_2.jpg
images/ZG521II_3.jpg
images/ZG7533AC_4.jpg
images/ZG707VK_5.jpg
images/ZG120EL_6.jpg
images/ZG2877J_7.jpg
images/ZG3585AI_8.jpg
images/ZG9267AD_9.jpg
images/SI661BM_10.jpg
images/CA890BC_11.jpg
images/IL111AT_12.jpg
images/ZG990NL_13.jpg
images/ZG367S0_14.jpg
images/ZG2563AC_15.jpg
images/ZG636RG_16.jpg
images/BEJ5345_17.jpg
images/ZG9952P_18.jpg
images/ZG877TF_19.jpg
images/ZG120JL_20.jpg
images/DU641CC_21.jpg
images/ZG8010AF_22.jpg
images/ZG401CC_23.jpg
images/KT451AN_24.jpg
images/ZG21010_25.jpg
images/G13GC_26.jpg
images/DJ944AE_27.jpg
images/ZG1908AA_28.jpg
images/MA356M_29.jpg
images/ZG400HZ_30.jpg
images/ZG5800K_31.jpg
images/0S001GS_32.jpg
images/ZG9158AK_33.jpg
images/SI582AM_34.jpg
images/ZGNIVEA1_35.jpg
images/SB268BZ_36.jpg
images/ZG932UG_37.jpg
images/VU936E_38.jpg
images/ZG256FZ_39.jpg
images/ZG784FU_40.jpg
images/ZG715FB_41.jpg
images/ZG655SC_42.jpg
images/983T939_43.jpg
images/KC0010KK_44.jpg
images/ZG1720AJ_45.jpg
images/DU879B

<ipython-input-9-579615bd4155>:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 1
<ipython-input-9-579615bd4155>:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 0


images/DU900CF_97.jpg
images/ZG093PZ_98.jpg
images/W21337S_99.jpg
images/ZG636TT_100.jpg
images/ZG624K_101.jpg
images/PFS3682_102.jpg
images/DU732Ch_103.jpg
images/KA570CD_104.jpg
images/S233IU_105.jpg
images/ZG5572AB_106.jpg
images/VU253H_107.jpg
images/DU993V_108.jpg
images/ZG424GH_109.jpg
images/ZG9492Z_110.jpg
images/010K414_111.jpg
images/KA555ZG_112.jpg
images/ZG1920T_113.jpg
images/BH633ZE_114.jpg
images/ZG41340_115.jpg
images/ZG270JB_116.jpg
images/SI124BP_117.jpg
images/ZG5114E_118.jpg
images/ZG316GB_119.jpg
images/RI681GN_120.jpg
images/ZG314U_121.jpg
images/GPJ442_122.jpg
images/LJT731F_123.jpg
images/ZD448AK_124.jpg
images/VU279AE_125.jpg
images/VU369AN_126.jpg
images/ZG394JU_127.jpg
images/PU253IB_128.jpg
images/ZG6347H_129.jpg
images/RI883FV_130.jpg
images/ZG6123J_131.jpg
images/KK2313L_132.jpg
images/PU865HZ_133.jpg
images/ZG841ZL_134.jpg
images/SI663BN_135.jpg
images/ZG20920_136.jpg
images/018050A_137.jpg
images/FGH1662_138.jpg
images/VU133C_139.jpg
images/VU936E_140.jp

### ICDar to YoLOv5 conversion

In [5]:
import xml.etree.ElementTree as ET
import pickle
import os
import cv2
from os import listdir, getcwd
from os.path import join


def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (round(x,5),round(y,5),round(w,5),round(h,5))

In [6]:
aug_txt_folder = 'dalabel_detection/'
aug_img_folder = 'daimg_detection/'
Yolo_txt_folder = 'Y_dalabel/'
dalabel_list = [file for file in os.listdir(aug_txt_folder) if file.endswith('.txt')]
print(len(dalabel_list))

1800


In [7]:
if not os.path.exists(Yolo_txt_folder):
    os.mkdir(Yolo_txt_folder)

In [8]:
for i in dalabel_list:
    y_img=cv2.imread(aug_img_folder+i[:-4]+'.jpg')
    h, w = y_img.shape[:2]
    in_file = open(aug_txt_folder+i,'r')
    out_file = open(Yolo_txt_folder+i, 'w')
    line = in_file.read().splitlines()
    print(line)
    for obj1 in line:
        obj =obj1.split(' ')
        b = (float(obj[1]), float(obj[3]), float(obj[2]), float(obj[4]))
        bb = convert((w,h), b)
        out_file.write(str(0) + " " + " ".join([str(a) for a in bb]) + '\n')

['ZG4940L 222 291 453 338']
['ZG548TL 96 315 284 370']
['MA398AG 181 338 404 378']
['BJ286BM 234 370 391 403']
['ZG2877J 261 302 361 366']
['ZG521II 222 310 441 348']
['ZG7533AC 217 309 433 360']
['ZG707VK 206 273 436 322']
['ZG969LN 81 362 143 404']
['RI365FN 204 395 448 441']
['ZG1665AE 233 215 430 280']
['RI741CR 210 394 437 440']
['ZG093PZ 190 288 439 341']
['RI393BD 258 268 392 359']
['NG179AC 196 288 426 333']
['ZG966ZT 230 282 442 331']
['ZG637LV 246 304 366 379']
['SK402CK 143 334 316 369']
['ZG505CE 270 280 397 371']
['ZG120EL 208 304 440 345']
['ZG2877J 261 302 361 366']
['ZG210EH 208 309 430 350']
['ZG3585AI 230 324 429 363']
['SK195BK 300 201 496 285']
['SI334BM 236 306 450 352']
['SI569AD 196 269 437 318']
['BH633ZE 206 251 452 311']
['ZG9267AD 210 571 440 623']
['PU659GB 229 298 431 344']
['ZG887K 200 315 415 363']
['ZG5670I 209 302 456 347']
['ZG956RZ 211 255 455 305']
['ZD125AM 190 262 449 325']
['ZG093PZ 190 288 439 341']
['SI661BM 217 261 419 305']
['CA890BC 216 379 4

['ZG9074F 271 370 389 393']
['FAM165 211 359 418 402']
['ZG4515AH 271 398 453 434']
['ZG877TF 201 306 429 346']
['VU936E 267 366 337 384']
['ZG548TL 96 315 284 370']
['ZG8481L 131 299 300 337']
['MB4071P 208 291 448 341']
['ZG956RZ 211 255 455 305']
['ZG8152G 214 277 448 331']
['ZG163DB 219 257 445 308']
['ZG4097AC 246 358 342 420']
['BJ765BM 199 287 422 331']
['ZG2273E 330 480 574 545']
['NJA1473 201 359 386 399']
['ZG461DJ 202 229 382 341']
['0G392A 216 279 492 342']
['ZG3833AK 195 409 440 464']
['HAD677 313 348 687 442']
['ZG1455AL 215 285 435 337']
['ZG9043B 215 357 423 405']
['VK551AG 205 264 458 322']
['EZG26 208 385 417 425']
['ZG145BD 266 260 393 350']
['ZG911CK 226 400 429 432']
['RI545IC 211 386 436 433']
['ZG759RR 209 269 443 318']
['KA127C0 189 292 384 336']
['ZG884AM 189 273 393 315']
['DU900CF 206 282 415 334']
['ZG093PZ 190 288 439 341']
['SI610AK 223 226 436 278']
['ZG594TS 213 251 434 306']
['ZG8298F 215 368 438 410']
['ZG8405A 233 352 432 394']
['W21337S 195 288 418 3

['ZG977SI 348 443 692 522']
['RI365FN 204 395 448 441']
['ZG415AC 199 260 461 313']
['BJ286BM 234 370 391 403']
['ZG416PZ 360 412 648 474']
['KEHH241 275 380 354 405']
['ZG911CK 226 400 429 432']
['ZG163DB 218 293 447 338']
['ZG668IC 285 345 401 417']
['ZG270JB 255 300 380 377']
['RI949GE 214 255 444 301']
['ZG443TP 207 296 422 343']
['ZG2448D 382 347 589 395']
['ZG2462AE 122 325 286 362']
['066A007 186 258 384 314']
['KZ386L 212 242 426 295']
['ZG160DU 220 304 416 344']
['KT451AN 205 290 443 339']
['ZG427TU 202 204 441 266']
['PU865HZ 209 299 425 351']
['SI124BP 195 288 428 333']
['ZG9074F 271 370 389 393']
['PU659GB 229 298 431 344']
['ZG655SC 213 401 431 447']
['ZD125AM 190 262 449 325']
['BIRM160 250 233 469 286']
['ZG155UB 218 252 451 307']
['ZG9572J 222 386 450 432']
['ZG784FU 213 309 449 355']
['ZG887K 200 315 415 363']
['NG619AA 169 280 381 319']
['LBVM427 205 253 439 304']
['ZG710KS 215 300 443 346']
['ZG636RG 209 209 395 270']
['KA127C0 189 292 384 336']
['RI545IC 211 386 436

['ST377IE 357 342 660 422']
['ZG6106I 194 250 413 304']
['ZG715FB 239 251 391 354']
['ZG2448D 382 347 589 395']
['ZG1628AK 212 272 437 318']
['0G788P 216 346 434 391']
['ZG5114E 265 371 379 397']
['ZG160DU 220 304 416 344']
['KAAC1689 199 256 457 325']
['ZG2751B 121 294 289 335']
['ZS38584 204 260 469 319']
['ST317KJ 201 270 445 314']
['ZG5642AB 198 284 456 342']
['VU727AK 281 262 364 289']
['KR401A 223 285 451 329']
['SPG065 220 341 411 388']
['ZG4256AI 218 259 400 301']
['ZG3209F 254 235 390 326']
['RI982JF 237 298 494 350']
['RI580KC 198 251 457 308']
['010K414 199 246 452 309']
['ZG4459L 226 366 422 404']
['ZG41340 205 289 415 333']
['A984KR 274 385 349 408']
['ZG4215AC 192 246 433 295']
['ZG3209F 254 235 390 326']
['VU3003VU 213 370 432 413']
['ZG155UB 218 252 451 307']
['ZG542VB 346 384 646 458']
['ZG155MC 196 244 437 307']
['ZG20920 212 234 483 292']
['DU900CF 206 282 415 334']
['ZG4949S 210 283 431 328']
['VU279AE 118 317 189 342']
['ZG900D0 200 287 435 328']
['ZG375UJ 252 369 

['ZG754VV 210 245 407 291']
['ZG7499AJ 211 258 450 307']
['KR401A 223 285 451 329']
['042K729 199 259 464 317']
['FGH1662 640 665 1169 843']
['ZG394JU 200 272 436 316']
['RI393BD 258 268 392 359']
['KK2313L 181 234 462 297']
['SK195BK 300 201 496 285']
['GPJ442 241 195 380 283']
['ZG2328V 224 260 419 314']
['ZG794VM 267 386 356 409']
['ZG885PK 214 271 452 324']
['KC620AL 199 291 433 339']
['ZG6983AG 267 383 391 410']
['ZG4256AI 218 259 400 301']
['VK551AG 205 264 458 322']
['513J636 212 217 437 273']
['ZG9023AI 263 352 378 377']
['ZG424GH 568 640 967 744']
['ZG145BD 266 260 393 350']
['ZG4949S 210 283 431 328']
['ZG877TF 201 306 429 346']
['ZG248EB 222 375 426 415']
['ZG4851T 215 266 429 316']
['KZ425V 211 271 450 329']
['VT769E 114 370 233 398']
['ZG4513R 202 309 420 355']
['GS971AI 108 318 300 358']
['ZG6106I 194 250 413 304']
['RI580KC 198 251 457 308']
['ZG162LC 259 278 370 356']
['ZG727LI 200 383 454 428']
['KA127C0 189 292 384 336']
['ZG1455AL 215 285 435 337']
['LN608AC 217 237 

['G13GC 206 259 422 311']
['ZG160DU 220 304 416 344']
['ST937IS 171 216 347 299']
['ZG655SC 213 401 431 447']
['GS971AI 108 318 300 358']
['ZG862NS 204 254 455 307']
['DA545AI 211 294 449 342']
['BA434DD 262 231 433 291']
['ZG40800 298 204 484 287']
['W78512C 314 559 657 649']
['ZG2808AA 363 346 651 419']
['ZG9158AK 218 382 424 426']
['VU133C 200 285 447 336']
['ZG6347H 181 241 460 302']
['ZG862NS 210 323 435 370']
['ZG9572J 222 386 450 432']
['ZG6123J 231 351 350 429']
['RI883FV 169 251 413 314']
['ZG9572J 222 386 450 432']
['ZG424RI 216 325 458 384']
['DU986BM 395 396 676 463']
['RI278IV 192 210 405 262']
['SI661BM 217 261 419 305']
['ZG394JU 200 272 436 316']
['ZG789PI 216 413 435 452']
['KKH5122 221 324 429 365']
['2216YE06 216 230 410 272']
['ZG441AJ 207 282 445 324']
['785K686 193 258 445 314']
['ST174IC 215 263 460 317']
['RI238JL 360 583 669 652']
['ZG051DM 193 362 401 432']
['KA596CJ 190 278 430 329']
['AHV9002 201 268 443 319']
['ZG7949AD 216 380 426 420']
['BJ100EE 170 325 3

In [ ]:
for i in dalabel_list:
    y_img=cv2.imread(aug_img_folder+i[:-4]+'.jpg')
    h, w = y_img.shape[:2]
    in_file = open(aug_txt_folder+i,'r')
    out_file = open(Yolo_txt_folder+i, 'w')
    line = in_file.read().splitlines()
    string ='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ-'
    str_list=list(str_list)
    print(len(string))
    print(line)
    for obj1 in line:
        obj =obj1.split(' ')
        b = (float(obj[1]), float(obj[3]), float(obj[2]), float(obj[4]))
        bb = convert((w,h), b)
        str_val = str_list.index(str(obj[0]))
        out_file.write(str(str_val) + " " + " ".join([str(a) for a in bb]) + '\n')

## convert voc to yolov5

In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (round(x,5),round(y,5),round(w,5),round(h,5))

def convert_annotation(image_id):
    in_file = open('label/%s.xml'%(image_id))
    out_file = open('labels/%s.txt'%( image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(0) + " " + " ".join([str(a) for a in bb]) + '\n')

In [ ]:
if not os.path.exists('labels/'):
    os.makedirs('labels/')
image_ids = [file for file in os.listdir('JPEGImages/') if file.endswith('.jpg')]
list_file = open('trainval.txt', 'w')
for image_ids in image_ids:
    image_id = image_ids[:-4]
    list_file.write('JPEGImages/%s.jpg\n'%(image_id))
    convert_annotation(image_id)
list_file.close()

##### some more things to add for boosting performance of object detection
#https://blog.gofynd.com/boost-object-detection-model-accuracy-552586d698c

In [ ]:
#JPEGImages/GPJ442_122.jpg

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


def unwarp(img, src, dst, testing):
    h, w = img.shape[:2]
    # use cv2.getPerspectiveTransform() to get M, the transform matrix, and Minv, the inverse
    M = cv2.getPerspectiveTransform(src, dst)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, (w, h), flags=cv2.INTER_LINEAR)

    if testing:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        f.subplots_adjust(hspace=.2, wspace=.05)
        ax1.imshow(img)
        x = [src[0][0], src[2][0], src[3][0], src[1][0], src[0][0]]
        y = [src[0][1], src[2][1], src[3][1], src[1][1], src[0][1]]
        ax1.plot(x, y, color='red', alpha=0.4, linewidth=3, solid_capstyle='round', zorder=2)
        ax1.set_ylim([h, 0])
        ax1.set_xlim([0, w])
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(cv2.flip(warped, 1))
        ax2.set_title('Unwarped Image', fontsize=30)
        plt.show()
    else:
        return warped, M


im = cv2.imread("GPJ442_122.jpg")

w, h = im.shape[0], im.shape[1]
# We will first manually select the source points 
# we will select the destination point which will map the source points in
# original image to destination points in unwarped image
src = np.float32([(20,     1),
                  (540,  130),
                  (20,    520),
                  (570,  450)])

dst = np.float32([(600, 0),
                  (0, 0),
                  (600, 531),
                  (0, 531)])

unwarp(im, src, dst, True)


In [ ]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('GPJ442_122.jpg')
rows,cols,ch = img.shape

pts1 = np.float32([[360,50],[2122,470],[2264, 1616],[328,1820]])

ratio=1.6
cardH=math.sqrt((pts1[2][0]-pts1[1][0])*(pts1[2][0]-pts1[1][0])+(pts1[2][1]-pts1[1][1])*(pts1[2][1]-pts1[1][1]))
cardW=ratio*cardH;
pts2 = np.float32([[pts1[0][0],pts1[0][1]], [pts1[0][0]+cardW, pts1[0][1]], [pts1[0][0]+cardW, pts1[0][1]+cardH], [pts1[0][0], pts1[0][1]+cardH]])

M = cv2.getPerspectiveTransform(pts1,pts2)

offsetSize=500
transformed = np.zeros((int(cardW+offsetSize), int(cardH+offsetSize)), dtype=np.uint8);
dst = cv2.warpPerspective(img, M, transformed.shape)

plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

# contrast

In [ ]:
import cv2
import numpy as np

# Open a typical 24 bit color image. For this kind of image there are
# 8 bits (0 to 255) per color channel
img = cv2.imread('mandrill.png')  # mandrill reference image from USC SIPI

s = 128
img = cv2.resize(img, (s,s), 0, 0, cv2.INTER_AREA)

def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [ ]:
import cv2

image = cv2.imread('GPJ442_122.jpg')

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# cv2.imshow('original', image)
# cv2.imshow('adjusted', adjusted)
# cv2.waitKey()

In [ ]:
import cv2
import numpy as np
image = cv2.imread("GPJ442_122.jpg")

alpha=1.5
beta=20

new_image=cv2.addWeighted(image,alpha,np.zeros(image.shape, image.dtype),0,beta)

# cv2.imshow("new",new_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## apply vertical and horizental motion blur
#https://stackoverflow.com/questions/40305933/how-to-add-motion-blur-to-numpy-array

In [ ]:
import cv2
import numpy as np
from random import randrange, uniform

# randrange gives you an integral value
size = randrange(1, 5)
print(size)
img = cv2.imread('GPJ442_122.jpg')
cv2.imshow('Original', img)

# generating the kernel
kernel_motion_blur = np.zeros((size, size))
kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
kernel_motion_blur = kernel_motion_blur / size

# applying the kernel to the input image
output = cv2.filter2D(img, -1, kernel_motion_blur)

cv2.imshow('Motion Blur', output)
cv2.waitKey(0)

In [ ]:
import cv2
import numpy as np

#size - in pixels, size of motion blur
#angel - in degrees, direction of motion blur
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )  
    k = k * ( 1.0 / np.sum(k) )        
    return cv2.filter2D(image, -1, k)

### voc data batch processing for changing label name

In [ ]:
import os
import xml.etree.ElementTree as ET

# : Batch modify the label name of the xml tag file in the VOC data set
def changelabelname(inputpath):
    listdir = os.listdir(inputpath)
    for file in listdir:
        if file.endswith('xml'):
            file = os.path.join(inputpath,file)
            tree = ET.parse(file)
            root = tree.getroot()
            for object1 in root.findall('object'):
                for sku in object1.findall('name'):           #Find the name you need to modify
                    if (sku.text == sku.text):               #‘preName’ is the name before the modification
                        sku.text = 'route'                 #‘TESTNAME’ is the modified name
                        tree.write(file,encoding='utf-8')     #Write into the original xml file and avoid the original xml Chinese characters garbled
                    else:
                        pass
        else:
            pass

if __name__ == '__main__':
    inputpath = 'label'  # 
    changelabelname(inputpath)